In [11]:
# https://es.stackoverflow.com/questions/32165/qu%C3%A9-es-if-name-main
if __name__ == '__main__':
    import os
    from pprint import pprint

    from msl.equipment import EquipmentRecord, ConnectionRecord, Backend

    from msl.equipment.resources.thorlabs import MotionControl
    # ensure that the Kinesis folder is available on PATH
    os.environ['PATH'] += os.pathsep + 'C:/Program Files/Thorlabs/Kinesis'

    print('Building the device list...')
    MotionControl.build_device_list()
    
    # rather than reading the EquipmentRecord from a database we can create it manually
    record = EquipmentRecord(
        manufacturer='Thorlabs',
        model='KST101',
        serial='26001683',  # update the serial number for your KST101
        connection=ConnectionRecord(
            backend=Backend.MSL,
            address='SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll',
        ),
    )

Building the device list...


In [16]:
print('Building the device list...')
MotionControl.build_device_list()

record2 = EquipmentRecord(
    manufacturer='Thorlabs',
    model='KST101',
    serial='26001749',  # update the serial number for your KST101
    connection=ConnectionRecord(
        backend=Backend.MSL,
        address='SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll',
    ),
)

Building the device list...


In [17]:
def wait(value,motor):
        motor.clear_message_queue()
        message_type, message_id, _ = motor.wait_for_message()
        while message_type != 2 or message_id != value:
            position = motor.get_position()
            real = motor.get_real_value_from_device_unit(position, 'DISTANCE')
            print('  at position {} [device units] {} [real-world units]'.format(position, real))
            message_type, message_id, _ = motor.wait_for_message()

In [18]:
motory = record2.connect()
print('Connected to {}'.format(motory))
# connect to the KCube Stepper Motor
motorx = record.connect()
print('Connected to {}'.format(motorx))

Cannot find <Device Name="26001749"> in ThorlabsDeviceConfiguration.xml


Connected to KCubeStepperMotor<Thorlabs|KST101|26001749>
Connected to KCubeStepperMotor<Thorlabs|KST101|26001683>


In [20]:
  # load the configuration settings (so that we can use the get_real_value_from_device_unit() method)

motory.load_settings()
motorx.load_settings()

KCubeStepperMotor<Thorlabs|KST101|26001749 at SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll> The function did not return True.
KCubeStepperMotor.SCC_LoadSettings(b'26001749',) -> False


ThorlabsError: KCubeStepperMotor<Thorlabs|KST101|26001749 at SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll>
The function did not return True.
KCubeStepperMotor.SCC_LoadSettings(b'26001749',) -> False

In [22]:
motory.disconnect()
motorx.disconnect()


In [8]:
def move_to_x_y_position(x,y):
    motorx.move_to_position(motorx.get_device_unit_from_real_value(x,0))
    wait(1,motorx)
    print('Moving done. At position {} [device units]'.format(motorx.get_position()))
    motory.move_to_position(motory.get_device_unit_from_real_value(y,0))
    wait(1,motory)
    print('Moving done. At position {} [device units]'.format(motory.get_position()))

In [9]:
move_to_x_y_position(10.0,10.0)

Moving done. At position 0 [device units]
Moving done. At position 0 [device units]
